<a href="https://colab.research.google.com/github/Mallesh06/Smart-Rain-Detection-Automated-Irrigation/blob/main/copy_of_smart_rain_detection_and_pump_control_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch torchvision datasets pillow


In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
import torch


In [ ]:
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=3) # Changed num_labels to 3

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/Colab Notebooks/dataset")

# Verify
print(dataset)


Resolving data files:   0%|          | 0/468 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/298 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 468
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 298
    })
})


In [ ]:
normalize = transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize,
])


In [ ]:
training_args = TrainingArguments(
    output_dir="./vit-finetuned-rain",
    per_device_train_batch_size=8,
    eval_strategy="steps",
    num_train_epochs=3,  # You can increase to 5–10 if you have GPU time
    save_steps=250,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-5,
    push_to_hub=False,
    report_to="none",  # Disable Weights & Biases logging
)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
def preprocess_image(examples):
    # Apply the defined transformations to the 'image' column
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    # Return only the processed pixel values and the labels
    return {"pixel_values": examples["pixel_values"], "label": examples["label"]}

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_image, batched=True)

# Remove the original 'image' column as it's no longer needed
processed_dataset = processed_dataset.remove_columns(["image"])

# Verify the structure of the processed dataset
print(processed_dataset)

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Map:   0%|          | 0/298 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'pixel_values'],
        num_rows: 468
    })
    test: Dataset({
        features: ['label', 'pixel_values'],
        num_rows: 298
    })
})


In [ ]:
unique_labels = set()

for split in processed_dataset:
    for example in processed_dataset[split]:
        unique_labels.add(example['label'])

print("Unique labels found in the dataset:", unique_labels)

Unique labels found in the dataset: {0, 1, 2}


In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Accuracy
10,1.011700,1.108179,0.345638
20,0.837400,1.079766,0.587248
30,0.661800,1.054914,0.684564
40,0.529200,1.033011,0.711409
50,0.429400,0.973987,0.724832
60,0.298100,0.928527,0.724832
70,0.228900,0.902084,0.724832
80,0.177600,0.885502,0.724832
90,0.141700,0.884342,0.724832
100,0.116100,0.896525,0.724832


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument i

TrainOutput(global_step=177, training_loss=0.28730123527979445, metrics={'train_runtime': 6436.2578, 'train_samples_per_second': 0.218, 'train_steps_per_second': 0.028, 'total_flos': 1.0879972854770074e+17, 'train_loss': 0.28730123527979445, 'epoch': 3.0})

In [ ]:
from PIL import Image
import torch

image = Image.open("/content/drive/MyDrive/Colab Notebooks/cloudy6.jpg").convert("RGB")
inputs = feature_extractor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.nn.functional.softmax(outputs.logits, dim=-1)
    label = preds.argmax(-1).item()

labels = ["Rainy", "Not_Rainy"]
print(f"Prediction: {labels[label]}")


Prediction: Rainy


In [ ]:
if labels[label] == "Rainy":
    print("💧 Rain detected → Stopping irrigation pump.")
else:
    print("☀️ No rain detected → Pump remains ON.")


💧 Rain detected → Stopping irrigation pump.


In [ ]:
model.save_pretrained("/content/vit-rain-model")


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("MALLESH06/smart-rain-detection")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...w34c_t4/model.safetensors:   3%|2         | 9.95MB /  343MB            

CommitInfo(commit_url='https://huggingface.co/MALLESH06/smart-rain-detection/commit/19c21c0082d71f2ea1ab6d5f6df24de29a060a82', commit_message='Upload ViTForImageClassification', commit_description='', oid='19c21c0082d71f2ea1ab6d5f6df24de29a060a82', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MALLESH06/smart-rain-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='MALLESH06/smart-rain-detection'), pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained("/content/vit-rain-model")

In [ ]:
import nbformat

# Load the notebook
with open("/content/drive/MyDrive/Colab Notebooks/Smart Rain Detection and Pump Control System.ipynb", "r") as f:
    notebook = nbformat.read(f, as_version=4)

# Remove the 'widgets' key from the metadata if it exists
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']

# Save the modified notebook
with open("/content/drive/MyDrive/Colab Notebooks/Smart Rain Detection and Pump Control System.ipynb", "w") as f:
    nbformat.write(notebook, f)

print("Notebook metadata updated. You can now try pushing to GitHub again.")

Notebook metadata updated. You can now try pushing to GitHub again.


In [ ]:
!git clone https://github.com/Mallesh06/Smart-Rain-Detection-Automated-Irrigation.git
!ls Smart-Rain-Detection-Automated-Irrigation/*.ipynb

Cloning into 'Smart-Rain-Detection-Automated-Irrigation'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 33 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 30.02 KiB | 3.75 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Smart-Rain-Detection-Automated-Irrigation/Smart_Rain_Detection_and_Pump_Control_System.ipynb
Smart-Rain-Detection-Automated-Irrigation/smart_rain_detection.ipynb
